In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

folder_path = "/content/drive/MyDrive/Nour/twitter"
print(f"Working with folder: {folder_path}")

Working with folder: /content/drive/MyDrive/Nour/twitter


In [ ]:
# Paths
dir = "/content/drive/MyDrive/"
output_dir = "/content/drive/MyDrive/Nour"
date = "june_2020"
merged_csv = f"{output_dir}/dedup_billgates_tweets_en_{date}.csv"
os.makedirs(output_dir, exist_ok=True)

# **twitter data extraction**

In [ ]:
!pip install pandas-dedupe

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.3/347.3 kB 21.0 MB/s 

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import re
import os

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths
dir = "/content/drive/MyDrive/"
output_dir = "/content/drive/MyDrive/Nour"
date = "june_2020"
merged_csv = f"{output_dir}/dedup_billgates_tweets_en_{date}.csv"
os.makedirs(output_dir, exist_ok=True)

# Clean text
BAD = [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
BAD_UNICODE_CHARS = [chr(b) for b in BAD]

def remove_bad_chars(value):
    for char in BAD_UNICODE_CHARS:
        value = value.replace(char, u' ')
    return str(value)

def clean_sentence(text):
    text = re.sub(r'\n', ' ', text)
    text = remove_bad_chars(str(text))
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'@\S+', ' ', text)
    text = re.sub(r'#\S+', ' ', text)
    text = re.sub(r'\$\S+', ' ', text)
    text = re.sub(r'&\S+', ' ', text)
    return text.strip()

def process_parquet_to_csv(user_type, date):
    input_path = os.path.join(dir, f"{user_type}_{date}.parquet")
    output_csv = os.path.join(output_dir, f"{user_type}_cleaned_{date}.csv")

    parquet_file = pq.ParquetFile(input_path)

    for i in range(parquet_file.num_row_groups):
        rg_table = parquet_file.read_row_group(i)
        df = rg_table.to_pandas()

        # Filter and clean
        df = df[df['lang'] == 'en']
        df['text_en'] = df['text'].apply(clean_sentence)
        df['user_type'] = user_type
        df = df.drop_duplicates(subset=['text_en'])

        # Save or append to CSV
        df.to_csv(output_csv, mode='w' if i == 0 else 'a',
                  index=False, header=i == 0)

    return output_csv

# Step 1: Process and save each category individually
csv_bot = process_parquet_to_csv("bot", date)
csv_deleted = process_parquet_to_csv("deleted", date)
csv_suspended = process_parquet_to_csv("suspended", date)
csv_other = process_parquet_to_csv("other", date)




Mounted at /content/drive


/tmp/ipython-input-1-2102409668.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_en'] = df['text'].apply(clean_sentence)
/tmp/ipython-input-1-2102409668.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_type'] = user_type


In [ ]:
import pandas as pd
print("🔁 Merging all category CSVs...")
csv_bot = os.path.join(output_dir, f"bot_cleaned_june_2020.csv")
csv_deleted = os.path.join(output_dir, f"deleted_cleaned_june_2020.csv")
csv_suspended = os.path.join(output_dir, f"suspended_cleaned_june_2020.csv")
csv_other = os.path.join(output_dir, f"other_cleaned_june_2020.csv")
df_all = pd.concat([
    pd.read_csv(csv_bot),
    pd.read_csv(csv_deleted),
    pd.read_csv(csv_suspended),
    pd.read_csv(csv_other)
])

df_all = df_all.drop_duplicates(subset=['text_en'])
df_all.to_csv(merged_csv, index=False)

print(f"✅ All done! Final merged CSV at:\n{merged_csv}")

🔁 Merging all category CSVs...


/tmp/ipython-input-4-1508574566.py:8: DtypeWarning: Columns (6,11,15,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_bot),
/tmp/ipython-input-4-1508574566.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_deleted),
/tmp/ipython-input-4-1508574566.py:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_suspended),
/tmp/ipython-input-4-1508574566.py:11: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_other)


✅ All done! Final merged CSV at:
/content/drive/MyDrive/Nour/dedup_billgates_tweets_en_june_2020.csv


In [ ]:
df_all['user_id'].unique()

array(['IllinoisJack85', 'TheRISEofROD', 'GmanFan45', ..., 'VitaminSteve',
       'Kruzer60913071', 'marlahooch231'], dtype=object)

# **Twitter data preparation**

In [ ]:
import pandas as pd
df=pd.read_csv(merged_csv)

/tmp/ipython-input-6-3634439614.py:2: DtypeWarning: Columns (6,11,15,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(merged_csv)


In [ ]:
df.columns

Index(['id', 'tweet_url', 'created_at', 'user_screen_name', 'text',
       'tweet_type', 'coordinates', 'hashtags', 'media', 'urls',
       'favorite_count', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'lang', 'place', 'possibly_sensitive',
       'retweet_count', 'retweet_or_quote_id', 'retweet_or_quote_screen_name',
       'retweet_or_quote_user_id', 'source', 'user_id', 'user_created_at',
       'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified',
       'text_en', 'user_type'],
      dtype='object')

In [ ]:
df['lang'].unique()

array(['en'], dtype=object)

In [ ]:
df['user_id'].unique()

array(['illinoisjack85', 'thebaronsamadhi', 'epochchanger', ...,
       'roberta86681937', 'ish91lopez', 'paulmuaddib61'], dtype=object)

In [ ]:
df['tweet_type'].unique()

array(['original', 'retweet', 'quote', 'reply'], dtype=object)

In [ ]:
df['user_type'].unique()

array(['bot', 'deleted', 'suspended', 'other'], dtype=object)

In [ ]:
df['retweet_or_quote_id'].unique()

array([nan, '1260576861228236802', '1267840224446554112', ...,
       '1276225572277030917', '1267627673238614016',
       '1278006706027376641'], dtype=object)

In [ ]:
df['in_reply_to_screen_name'].unique()

array([nan, 'alaphiah', 'SmitsDerek', ..., 'lowrentkicker',
       'HeyThatsComics', 'jessycarayanne'], dtype=object)

In [ ]:
df['in_reply_to_status_id'].unique()

array([           nan, 1.26788593e+18, 1.26803459e+18, ...,
       1.27785084e+18, 1.27804871e+18, 1.27806515e+18])

In [ ]:
df['in_reply_to_user_id'].unique()

array([           nan, 1.97906380e+07, 8.93248497e+17, ...,
       1.50858450e+07, 1.15419066e+18, 2.17993250e+07])

In [ ]:
df['retweet_or_quote_screen_name'].unique()

array([nan, 'georgesoros', 'kc47059011', ..., 'JonEndean', 'NapervillePD',
       'JimLaHa'], dtype=object)

In [ ]:
df['retweet_or_quote_id'].unique()

array([nan, '1260576861228236802', '1267840224446554112', ...,
       '1276225572277030917', '1267627673238614016',
       '1278006706027376641'], dtype=object)

In [ ]:
df['retweet_or_quote_user_id'].unique()

array([           nan, 2.31252570e+07, 9.33089374e+17, ...,
       4.12988610e+07, 4.96830410e+07, 2.60575840e+07])

In [ ]:
import pandas as pd


df['is_reply'] = df['in_reply_to_status_id'].notnull()
df['is_retweet'] = df['tweet_type'].str.lower() == 'retweet'
df['is_quote'] = df['tweet_type'].str.lower() == 'quote'


df['rt.id'] = df['retweet_or_quote_id'].where(df['is_retweet'], 0)
df['rt.user.id'] = df['retweet_or_quote_user_id'].where(df['is_retweet'], 0)

df['quote.id'] = df['retweet_or_quote_id'].where(df['is_quote'], 0)
df['quote.user.id'] = df['retweet_or_quote_user_id'].where(df['is_quote'], 0)


In [ ]:
col_mapping = {
    'id': 'id',
    'created_at': 'created_at',
    'text': 'text',
    'lang': 'lang',
    'hashtags': 'entities.hashtags',
    'tweet_url': 'entities.urls',
    'in_reply_to_status_id': 'in_reply_to_status_id',
    'in_reply_to_user_id': 'in_reply_to_user_id',
    'retweet_or_quote_id': 'rt.id',
    'retweet_or_quote_user_id': 'rt.user.id',
    'user_id': 'user.id',
    'user_screen_name': 'user.screen_name',
    'user_name': 'user.name',
    'user_description': 'user.description',
    'user_location': 'user.location',
    'user_created_at': 'user.created_at',
    'user_verified': 'user.verified',
    'user_followers_count': 'user.followers_count',
    'user_friends_count': 'user.friends_count',
    'user_listed_count': 'user.listed_count',
    'user_favourites_count': 'user.favourites_count',
    'user_statuses_count': 'user.statuses_count',
    'user_default_profile_image': 'user.default_profile_image',
    'user_urls': 'user.url',
    'user_type': 'user.type'
}

df = df.rename(columns=col_mapping)

In [ ]:
df.columns

Index(['id', 'entities.urls', 'created_at', 'user.screen_name', 'text',
       'tweet_type', 'coordinates', 'entities.hashtags', 'media', 'urls',
       'favorite_count', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'lang', 'place', 'possibly_sensitive',
       'retweet_count', 'rt.id', 'retweet_or_quote_screen_name', 'rt.user.id',
       'source', 'user.id', 'user.created_at', 'user.default_profile_image',
       'user.description', 'user.favourites_count', 'user.followers_count',
       'user.friends_count', 'user.listed_count', 'user.location', 'user.name',
       'user.statuses_count', 'user_time_zone', 'user.url', 'user.verified',
       'text_en', 'user_type', 'is_reply', 'is_retweet', 'is_quote', 'rt.id',
       'rt.user.id', 'quote.id', 'quote.user.id'],
      dtype='object')

In [ ]:
df['is_reply'].unique()

array([False,  True])

In [ ]:
df['in_reply_to_user_id'].unique()

array([           nan, 1.97906380e+07, 8.93248497e+17, ...,
       1.50858450e+07, 1.15419066e+18, 2.17993250e+07])

In [ ]:
df['is_reply'].count()

np.int64(73496)

In [ ]:
df['in_reply_to_user_id'].count()

np.int64(73496)

In [ ]:
import pandas as pd
df['user.protected'] = False
df['user.default_profile'] = False
df['user.profile_image_url_https'] = ""
df['entities.user_mentions'] = ""

In [ ]:
df.columns

Index(['id', 'entities.urls', 'created_at', 'user.screen_name', 'text',
       'tweet_type', 'coordinates', 'entities.hashtags', 'media', 'urls',
       'favorite_count', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'lang', 'place', 'possibly_sensitive',
       'retweet_count', 'rt.id', 'retweet_or_quote_screen_name', 'rt.user.id',
       'source', 'user.id', 'user.created_at', 'user.default_profile_image',
       'user.description', 'user.favourites_count', 'user.followers_count',
       'user.friends_count', 'user.listed_count', 'user.location', 'user.name',
       'user.statuses_count', 'user_time_zone', 'user.url', 'user.verified',
       'text_en', 'user_type', 'is_reply', 'is_retweet', 'is_quote', 'rt.id',
       'rt.user.id', 'quote.id', 'quote.user.id', 'user.protected',
       'user.default_profile', 'user.profile_image_url_https',
       'entities.user_mentions'],
      dtype='object')

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

In [ ]:
import hashlib
def hash_to_int9(s: str) -> int:
    """
    Deterministically hash a string to a 9-digit integer (0–999,999,999).
    """
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()
    return int(h, 16) % (10**9)

In [ ]:
df['user.id'] = df['user.id'].apply(hash_to_int9)
print(df.head())

                    id                                      entities.urls  \
0  1267443579045232651  https://twitter.com/IllinoisJack85/status/1267...   
1  1267520253904510976  https://twitter.com/IllinoisJack85/status/1267...   
2  1267587387443593217  https://twitter.com/IllinoisJack85/status/1267...   
3  1267840580299694082  https://twitter.com/IllinoisJack85/status/1267...   
4  1267887896884908032  https://twitter.com/IllinoisJack85/status/1267...   

                       created_at user.screen_name  \
0  Mon Jun 01 13:11:11 +0000 2020   IllinoisJack85   
1  Mon Jun 01 18:15:52 +0000 2020   IllinoisJack85   
2  Mon Jun 01 22:42:38 +0000 2020   IllinoisJack85   
3  Tue Jun 02 15:28:44 +0000 2020   IllinoisJack85   
4  Tue Jun 02 18:36:45 +0000 2020   IllinoisJack85   

                                                text tweet_type coordinates  \
0  During all those protests people suddenly forg...   original         NaN   
1  I just saw CBSN doing a POSITIVE report on Hyd...  

In [ ]:
# Define the list of columns to keep
columns_to_keep = [
    "id",
    "text",
    "created_at",
    "lang",
    "entities.urls",
    "entities.user_mentions",
    "entities.hashtags",
    "user.id",
    "user.description",
    "user.location",
    "user.name",
    "user.screen_name",
    "user.url",
    "user.protected",
    "user.verified",
    "user.followers_count",
    "user.friends_count",
    "user.listed_count",
    "user.favourites_count",
    "user.statuses_count",
    "user.created_at",
    "user.profile_image_url_https",
    "user.default_profile",
    "user.default_profile_image",
    "is_retweet",
    "is_quote",
    "is_reply",
    "in_reply_to_user_id",
    "in_reply_to_status_id",
    "quote.id",
    "quote.user.id",
    "rt.id",
    "rt.user.id",
    'user.type'
]


df = df[columns_to_keep]


In [ ]:
df.columns

Index(['id', 'text', 'created_at', 'lang', 'entities.urls',
       'entities.user_mentions', 'entities.hashtags', 'user.id',
       'user.description', 'user.location', 'user.name', 'user.screen_name',
       'user.url', 'user.protected', 'user.verified', 'user.followers_count',
       'user.friends_count', 'user.listed_count', 'user.favourites_count',
       'user.statuses_count', 'user.created_at',
       'user.profile_image_url_https', 'user.default_profile',
       'user.default_profile_image', 'is_retweet', 'is_quote', 'is_reply',
       'in_reply_to_user_id', 'in_reply_to_status_id', 'quote.id',
       'quote.user.id', 'rt.id', 'rt.user.id', 'user.type'],
      dtype='object')

In [ ]:
df['user.id'].unique()

array([467108258, 691055830, 324604086, ..., 655271621, 162996575,
        84803842])

In [ ]:
csv_path = os.path.join(folder_path,"twitter_tsonduku.csv")
pd.DataFrame(df).to_csv(csv_path, index=False, encoding="utf-8")

# **data compression**(for tsundoku toolkit)

In [ ]:
input_file_path = os.path.join(folder_path, "twitter_tsonduku.csv")

In [ ]:
import pandas as pd
df_tsonduku=pd.read_csv(input_file_path)

/tmp/ipython-input-6-2254166780.py:2: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tsonduku=pd.read_csv(input_file_path)


In [ ]:
import os
import pandas as pd
import gzip
import json
from datetime import datetime

In [ ]:
df_tsonduku.head(3)

,id,text,created_at,lang,entities.urls,entities.user_mentions,entities.hashtags,user.id,user.description,user.location,...,user.default_profile_image,is_retweet,is_quote,is_reply,in_reply_to_user_id,in_reply_to_status_id,quote.id,quote.user.id,rt.id,rt.user.id
0,1267443579045232651,During all those protests people suddenly forg...,Mon Jun 01 13:11:11 +0000 2020,en,https://twitter.com/IllinoisJack85/status/1267...,NaN,NaN,467108258,"""Thou shalt not follow a multitude to do evil""...","Mattoon, Illinois",...,False,False,False,False,NaN,NaN,0.0,0.0,NaN,NaN
1,1267520253904510976,I just saw CBSN doing a POSITIVE report on Hyd...,Mon Jun 01 18:15:52 +0000 2020,en,https://twitter.com/IllinoisJack85/status/1267...,NaN,NaN,467108258,"""Thou shalt not follow a multitude to do evil""...","Mattoon, Illinois",...,False,False,False,False,NaN,NaN,0.0,0.0,NaN,NaN
2,1267587387443593217,Who died and made you king? https://t.co/Phi4R...,Mon Jun 01 22:42:38 +0000 2020,en,https://twitter.com/IllinoisJack85/status/1267...,NaN,NaN,467108258,"""Thou shalt not follow a multitude to do evil""...","Mattoon, Illinois",...,False,True,False,False,NaN,NaN,0.0,0.0,1260576861228236802,23125257.0


In [ ]:
df_tsonduku.columns

Index(['id', 'text', 'created_at', 'lang', 'entities.urls',
       'entities.user_mentions', 'entities.hashtags', 'user.id',
       'user.description', 'user.location', 'user.name', 'user.screen_name',
       'user.url', 'user.protected', 'user.verified', 'user.followers_count',
       'user.friends_count', 'user.listed_count', 'user.favourites_count',
       'user.statuses_count', 'user.created_at',
       'user.profile_image_url_https', 'user.default_profile',
       'user.default_profile_image', 'is_retweet', 'is_quote', 'is_reply',
       'in_reply_to_user_id', 'in_reply_to_status_id', 'quote.id',
       'quote.user.id', 'rt.id', 'rt.user.id'],
      dtype='object')

In [ ]:
import pandas as pd
import gzip
import json
from datetime import datetime

df_tsonduku['created_at_dt'] = pd.to_datetime(df_tsonduku['created_at'], utc=True)

/tmp/ipython-input-15-957527837.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tsonduku['created_at_dt'] = pd.to_datetime(df_tsonduku['created_at'], utc=True)


In [ ]:
df_tsonduku = df_tsonduku.set_index('created_at_dt', drop=False)

In [ ]:
print(df_tsonduku.head(10))

                                            id  \
created_at_dt                                    
2020-06-01 13:11:11+00:00  1267443579045232651   
2020-06-01 18:15:52+00:00  1267520253904510976   
2020-06-01 22:42:38+00:00  1267587387443593217   
2020-06-02 15:28:44+00:00  1267840580299694082   
2020-06-02 18:36:45+00:00  1267887896884908032   
2020-06-03 04:50:25+00:00  1268042332416475136   
2020-06-03 21:27:38+00:00  1268293289582768128   
2020-06-03 21:30:10+00:00  1268293925376319490   
2020-06-03 21:44:04+00:00  1268297423753039874   
2020-06-04 02:24:59+00:00  1268368120760152067   

                                                                        text  \
created_at_dt                                                                  
2020-06-01 13:11:11+00:00  During all those protests people suddenly forg...   
2020-06-01 18:15:52+00:00  I just saw CBSN doing a POSITIVE report on Hyd...   
2020-06-01 22:42:38+00:00  Who died and made you king? https://t.co/Phi4R...   

In [ ]:
string_cols = [
    'text', 'created_at', 'lang', 'entities.urls', 'entities.user_mentions', 'entities.hashtags',
    'user.description', 'user.location', 'user.name', 'user.screen_name', 'user.url',
    'user.created_at', 'user.profile_image_url_https'
]

int_cols = [
    'id', 'user.id', 'user.followers_count', 'user.friends_count', 'user.listed_count',
    'user.favourites_count', 'user.statuses_count', 'in_reply_to_user_id', 'in_reply_to_status_id',
    'quote.id', 'quote.user.id', 'rt.id', 'rt.user.id'
]


df_tsonduku[string_cols] = df_tsonduku[string_cols].fillna("")
for col in int_cols:
    df_tsonduku[col] = pd.to_numeric(df_tsonduku[col], errors="coerce").fillna(0).astype(int)

In [ ]:
print(df_tsonduku.head(10))

                                            id  \
created_at_dt                                    
2020-06-01 13:11:11+00:00  1267443579045232651   
2020-06-01 18:15:52+00:00  1267520253904510976   
2020-06-01 22:42:38+00:00  1267587387443593217   
2020-06-02 15:28:44+00:00  1267840580299694082   
2020-06-02 18:36:45+00:00  1267887896884908032   
2020-06-03 04:50:25+00:00  1268042332416475136   
2020-06-03 21:27:38+00:00  1268293289582768128   
2020-06-03 21:30:10+00:00  1268293925376319490   
2020-06-03 21:44:04+00:00  1268297423753039874   
2020-06-04 02:24:59+00:00  1268368120760152067   

                                                                        text  \
created_at_dt                                                                  
2020-06-01 13:11:11+00:00  During all those protests people suddenly forg...   
2020-06-01 18:15:52+00:00  I just saw CBSN doing a POSITIVE report on Hyd...   
2020-06-01 22:42:38+00:00  Who died and made you king? https://t.co/Phi4R...   

In [ ]:
# Group into 10‑minute bins and export JSONL gzip files
for ts, group in df_tsonduku.groupby(pd.Grouper(freq='10min', key='created_at_dt')):
    if group.empty:
        continue

    ts_str = ts.strftime('%Y%m%d%H%M')
    out_file = os.path.join(folder_path, 'tsundoku_tweets', f'tweets_{ts_str}.data.gz')

    with gzip.open(out_file, 'wt', encoding='utf-8') as f:
        group = group.drop(columns=['created_at_dt'])
        for record in group.to_dict(orient='records'):
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

    print(f'Wrote {len(group)} tweets to {out_file}')


Wrote 473 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010000.data.gz
Wrote 517 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010010.data.gz
Wrote 458 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010020.data.gz
Wrote 452 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010030.data.gz
Wrote 452 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010040.data.gz
Wrote 475 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010050.data.gz
Wrote 487 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010100.data.gz
Wrote 485 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010110.data.gz
Wrote 454 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010120.data.gz
Wrote 496 tweets to /content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010130.data.gz


In [ ]:
import gzip
import json

file_path = '/content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006302200.data.gz'

with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        tweet = json.loads(line)
        print(json.dumps(tweet, indent=2, ensure_ascii=False))
        if i >= 2:  # just show the first 3 tweets
            break


{
  "id": 1278085908068274176,
  "text": "It's almost 4am and I'm just about done with @CrimeJunkiePod's episode on #TamlaHorsford.They're going to reopen the case due to the public outcry and the thousands of people who signed the petition! Ahhh 🔥 https://t.co/j35NOxLjip",
  "created_at": "Tue Jun 30 22:00:00 +0000 2020",
  "lang": "en",
  "entities.urls": "https://twitter.com/StephanieOdessa/status/1278085908068274176",
  "entities.user_mentions": "",
  "entities.hashtags": "TamlaHorsford",
  "user.id": 493261567,
  "user.description": "Soul Artist,African core. Art about people of African descent my conduit. News, Health,Culture. Social Justice Art,truth not always pretty picture. Black ❤️",
  "user.location": "In the Spirit",
  "user.name": "Stephanie",
  "user.screen_name": "StephanieOdessa",
  "user.url": "",
  "user.protected": false,
  "user.verified": false,
  "user.followers_count": 101,
  "user.friends_count": 107,
  "user.listed_count": 1,
  "user.favourites_count": 2600,
 

In [ ]:
!ls -lh /content/drive/MyDrive/Nour/twitter/tsundoku_tweets | tail

-rw------- 1 root root 198K Jul  2 21:24 tweets_202006302220.data.gz
-rw------- 1 root root 193K Jul  2 21:24 tweets_202006302230.data.gz
-rw------- 1 root root 194K Jul  2 21:24 tweets_202006302240.data.gz
-rw------- 1 root root 210K Jul  2 21:24 tweets_202006302250.data.gz
-rw------- 1 root root 212K Jul  2 21:24 tweets_202006302300.data.gz
-rw------- 1 root root 212K Jul  2 21:24 tweets_202006302310.data.gz
-rw------- 1 root root 201K Jul  2 21:24 tweets_202006302320.data.gz
-rw------- 1 root root 213K Jul  2 21:24 tweets_202006302330.data.gz
-rw------- 1 root root 198K Jul  2 21:24 tweets_202006302340.data.gz
-rw------- 1 root root 202K Jul  2 21:24 tweets_202006302350.data.gz


In [ ]:
!zip -r /content/tsundoku_tweets.zip /content/drive/MyDrive/Nour/twitter/tsundoku_tweets

  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/ (stored 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010000.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010010.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010020.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010030.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010040.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010050.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010100.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010110.data.gz (deflated 0%)
  adding: content/drive/MyDrive/Nour/twitter/tsundoku_tweets/tweets_202006010120.data.gz (deflated 0%)

In [ ]:
from google.colab import files
files.download("/content/tsundoku_tweets.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>